In [139]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)

In [140]:
df.size

5460

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Purchase ID  780 non-null    int64  
 1   SN           780 non-null    object 
 2   Age          780 non-null    int64  
 3   Gender       780 non-null    object 
 4   Item ID      780 non-null    int64  
 5   Item Name    780 non-null    object 
 6   Price        780 non-null    float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.8+ KB


In [142]:
df2=df.copy()


## Player Count

In [143]:
#unique number of players
uniquePlayer=df.SN.nunique()
playerCount = pd.DataFrame()
playerCount["Total Players"]= [uniquePlayer]
playerCount

,Total Players
0,576


## Purchasing Analysis (Total)

In [144]:
uniqueItems = df2["Item ID"].nunique()

In [145]:
avgPrice = df2.Price.mean()
avgPriceD = "${:,.2f}".format(avgPrice)

In [146]:
totalRev = df2.Price.sum()
totalRevD= "${:,.2f}".format(totalRev)

In [147]:
totalPurch = df2["Purchase ID"].count()

In [148]:
PurchAnalysis = pd.DataFrame()
PurchAnalysis["Number of Unique Items"] = [uniqueItems]
PurchAnalysis["Average Price"] = [avgPriceD]
PurchAnalysis["Number of Purchases"] = [totalPurch]
PurchAnalysis["TotalRevenue"] = [totalRevD]
PurchAnalysis

,Number of Unique Items,Average Price,Number of Purchases,TotalRevenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

In [149]:
#df.Gender.value_counts()

In [150]:
cols=["SN", "Age", "Gender"]
players = df.loc[:,cols]
#players.head()

In [151]:
#droping the duplicates
players = players.drop_duplicates()
#players.info()

In [152]:
playercnts= players.Gender.value_counts()
playercnts
playerperc = players.Gender.value_counts()/len(players)

In [168]:
genderDemo = pd.concat([playercnts,playerperc],axis = 1)
genderDemo.columns = ['Total Count', 'Percentage of Players']
genderDemo['Percentage of Players' ] = genderDemo['Percentage of Players'].map('{:.2%}'.format)
genderDemo

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [154]:
cols=["SN", "Price", "Gender"]
players2 = df2.loc[:,cols]

In [155]:
totalperplayer = players2.groupby("SN").sum()

In [156]:
analysisGender = df2.groupby('Gender')
purchasecount= analysisGender['Purchase ID'].count()
avgprice = analysisGender[['Price']].mean()
totalValue = analysisGender[['Price']].sum()

dfPA = pd.concat([purchasecount,avgprice, totalValue],axis = 1)
dfPA .columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']
dfPA ['Avg Total Purchase Per Person']=dfPA['Total Purchase Value']/ genderDemo['Total Count']

dfPA ['Total Purchase Value' ] = dfPA['Total Purchase Value'].map('${:,.2f}'.format)
dfPA ['Avg Total Purchase Per Person' ] = dfPA['Avg Total Purchase Per Person'].map('${:,.2f}'.format)
dfPA['Average Purchase Price' ] = dfPA['Average Purchase Price'].map('${:,.2f}'.format)

dfPA

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [157]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 99999999]
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "34-39", "40+"]

In [158]:
df2["Age Group"] = pd.cut(df2.Age, bins, labels=group_labels, include_lowest=True)

In [159]:
#df.groupby("Age Group").Age.min()

In [160]:
cols = ["SN", "Age", "Gender", "Age Group"]
players= df2.loc[:, cols]

In [161]:
players = players.drop_duplicates().reset_index(drop=True)

In [162]:
playerCnt = players.groupby("Age Group").size()
playerPerc = players.groupby("Age Group").size() / len(players)

# https://stackoverflow.com/a/18062521
dfad = pd.concat([playerCnt, playerPerc], axis=1)
dfad.columns = ["Total Count", 'Percentage of Players']

dfad['Percentage of Players' ] = dfad['Percentage of Players'].map('{:.2%}'.format)

dfad

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
34-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [163]:
purchAge = df2.groupby('Age Group')
purchasecount= purchAge['Purchase ID'].count()
avgprice = purchAge[['Price']].mean()
totalValue = purchAge[['Price']].sum()

dfpage = pd.concat([purchasecount,avgprice, totalValue],axis = 1)
dfpage.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']
dfpage['Avg Total Purchase Per Person']=dfpage['Total Purchase Value']/ dfad['Total Count']

dfpage ['Total Purchase Value' ] = dfpage['Total Purchase Value'].map('${:,.2f}'.format)
dfpage ['Avg Total Purchase Per Person' ] = dfpage['Avg Total Purchase Per Person'].map('${:,.2f}'.format)
dfpage['Average Purchase Price' ] = dfpage['Average Purchase Price'].map('${:,.2f}'.format)

dfpage

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
34-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [164]:
topSpend = df2.groupby('SN')
purchasecount= topSpend['Purchase ID'].count()
avgprice = topSpend[['Price']].mean()
totalValue = topSpend[['Price']].sum()

dfts = pd.concat([purchasecount,avgprice, totalValue],axis = 1)
dfts.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']
dfts.sort_values(by= 'Total Purchase Value', ascending = False, inplace=True)

dfts ['Total Purchase Value' ] = dfts['Total Purchase Value'].map('${:,.2f}'.format)
dfts['Average Purchase Price' ] = dfts['Average Purchase Price'].map('${:,.2f}'.format)

dfts.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [165]:
cols = ["Item ID", "Item Name", "Price"]
PopItems= df2.loc[:, cols]

In [166]:
sumPopItem = df2.groupby(["Item ID", "Item Name"])
purchasecount= sumPopItem['Item ID'].count()
avgprice = sumPopItem[['Price']].mean()
totalValue = sumPopItem[['Price']].sum()

dfpopI = pd.concat([purchasecount,avgprice, totalValue],axis = 1)
dfpopI.columns = ['Purchase Count', 'Item Price', 'Total Purchase Value']
dfpopI.sort_values(by= 'Purchase Count', ascending = False, inplace=True)

dfpopI['Item Price' ] = dfpopI['Item Price'].map('${:,.2f}'.format)
dfpopI['Total Purchase Value' ] = dfpopI['Total Purchase Value'].map('${:,.2f}'.format)

dfpopI.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

In [167]:
sumPopItem = df2.groupby(["Item ID", "Item Name"])
purchasecount= sumPopItem['Item ID'].count()
avgprice = sumPopItem[['Price']].mean()
totalValue = sumPopItem[['Price']].sum()

dfprofi = pd.concat([purchasecount,avgprice, totalValue],axis = 1)
dfprofi.columns = ['Purchase Count', 'Item Price', 'Total Purchase Value']
dfprofi.sort_values(by= 'Total Purchase Value', ascending = False, inplace=True)

dfprofi['Item Price' ] = dfprofi['Item Price'].map('${:,.2f}'.format)
dfprofi['Total Purchase Value' ] = dfprofi['Total Purchase Value'].map('${:,.2f}'.format)

dfprofi.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
